# Examining Gray's River streamflow quality flags

In [1]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
import ipywidgets as widgets
from ipywidgets import interact
from natsort import natsorted
import os
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib

In [2]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 140

#### Compile raw table files from Dept. of Ecology website into one CSV
https://fortress.wa.gov/ecy/eap/flows/station.asp?sta=25B060#block2


In [13]:
# grays_dir = config.streamflow.parents[0] / 'grays_river'
# # all_files = natsorted(next(os.walk(grays_dir))[2])

# txts1 = []
# for file in ['25B060_2006_DSG_DV.txt', '25B060_2007_DSG_DV.txt', '25B060_2008_DSG_DV.txt', '25B060_2009_DSG_DV.txt', '25B060_2010_DSG_DV.txt']:
#     grays_flow = pd.read_csv(grays_dir / file, delim_whitespace=True, skiprows=2, names=['date', 'time', 'flow (cfs)', 'quality'])
#     txts1.append(grays_flow)

# txts2 = []
# for file in ['25B060_2011_DSG_DV.txt', '25B060_2012_DSG_DV.txt']:
#     grays_flow = pd.read_csv(grays_dir / file, delim_whitespace=True, skiprows=2, names=['date', 'flow (cfs)', 'quality'])
#     txts2.append(grays_flow)

# df1 = pd.concat(txts1)
# df1.drop('time', inplace=True, axis=1)
# df2 = pd.concat(txts2)

# df = pd.concat([df1, df2])
# df.to_csv(grays_dir / '25B060_grays_river_05_12_flow.csv', index=False)

In [5]:
grays_dir = config.streamflow.parents[0] / 'grays_river'
grays_df = pd.read_csv(grays_dir / '25B060_grays_river_05_12_flow.csv', parse_dates=True, index_col=0)
grays_df['doy'], grays_df['year'] = grays_df.index.dayofyear, grays_df.index.year
grays_flow = grays_df.drop('quality', axis=1)
grays_yearly = pd.pivot_table(grays_flow, index=['doy'], columns=['year'], values='flow (cfs)')

In [10]:
# Add quality scores below plot

colors = sns.color_palette('tab10', 10)

quality = grays_df.drop('flow (cfs)', axis=1)
quality_edit = quality.copy()

quality_edit['doy'], quality_edit['year'] = quality_edit.index.dayofyear, quality_edit.index.year
quality_edit_yearly = pd.pivot_table(quality_edit, index=['doy'], columns=['year'], values=['quality'])
quality2 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [-2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality3 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality8 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                       [np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality10 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
quality50 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan, np.nan])
quality77 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                        [np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan, np.nan])
quality160 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan, np.nan])
quality161 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan, np.nan])
quality179 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2, np.nan])
quality254 = quality_edit_yearly.replace([2, 3, 8, 10, 50, 77, 160, 161, 179, 254],
                                         [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, -2])

years = grays_yearly.columns
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 14))

for col, year in enumerate(years):
#     ax2 = axes[col].twinx()
#     precip_yearly.iloc[:, col].plot(ax=ax2, label='Precip', color='tab:green', linewidth=0.5)
#     if col == 0:
#         ax2.set_ylabel('Precipitation (mm/day)')
#         ax2.legend(loc='upper right', bbox_to_anchor=(1, 1.3), fancybox=True, ncol=1)
    quality2.iloc[:, col].plot(ax=axes[col], color=colors[0], linewidth=5, label='Good provisional data')
    quality3.iloc[:, col].plot(ax=axes[col], color=colors[1], linewidth=5, label='Good provisional data - edited')
    quality8.iloc[:, col].plot(ax=axes[col], color=colors[2], linewidth=5, label='Below rating')
    quality10.iloc[:, col].plot(ax=axes[col], color=colors[3], linewidth=5, label='Above rating (<2x)')
    quality50.iloc[:, col].plot(ax=axes[col], color=colors[4], linewidth=5, label='Estimated')
    quality77.iloc[:, col].plot(ax=axes[col], color=colors[5], linewidth=5, label='Estimated from other station')
    quality160.iloc[:, col].plot(ax=axes[col], color=colors[6], linewidth=5, label='Above rating (>2x)')
    quality161.iloc[:, col].plot(ax=axes[col], color=colors[7], linewidth=5, label='Below rating (<1/2x)')
    quality179.iloc[:, col].plot(ax=axes[col], color=colors[8], linewidth=5, label='Estimated - unreliable')
    quality254.iloc[:, col].plot(ax=axes[col], color=colors[9], linewidth=5, label='Rating exceeded')
    grays_yearly.iloc[:, col].plot(ax=axes[col], label='Observed', linewidth=1)
#     ax2.invert_yaxis()
    axes[col].set_ylim([-5, 20000])
    axes[col].set_title(year)
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.9), fancybox=True, ncol=2)
axes[0].set_ylabel('Runoff (mm/day)')
plt.tight_layout()
# plt.savefig('baseline_runoffs_qaflags.png', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …